In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import time

# Create the bot
mineflayer = require('mineflayer')

random_number = id([]) % 1000
BOT_USERNAME = f'poo_bucket{random_number}'

bot = mineflayer.createBot({ 
    'host': 'localhost',
    'port': 22222,
    'username': BOT_USERNAME, 
    'hideErrors': False 
})

once(bot, 'login')
bot.chat('I spawned')

Vec3 = require('vec3').Vec3

# Load plugins
bot.loadPlugin(require('mineflayer-collectblock').plugin)

bot.loadPlugin(require('minecrafthawkeye'))

pathfinder = require('mineflayer-pathfinder')
bot.loadPlugin(pathfinder.pathfinder)

pvp = require('mineflayer-pvp').plugin
bot.loadPlugin(pvp)

mcData = require('minecraft-data')(bot.version)
globalThis.mcData = mcData

movements = pathfinder.Movements(bot, mcData)
bot.pathfinder.setMovements(movements)
globalThis.pathfinder = pathfinder

# Global range goal for movement
RANGE_GOAL = 1

bot.removeAllListeners('chat')
@On(bot, 'chat')
def handleMsg(this, sender, message, *args):
  if sender and (sender != BOT_USERNAME):
    bot.chat('Hi, you said ' + message)
    if 'come' in message:
      player = bot.players[sender]
      target = player.entity
      if not target:
        bot.chat("I don't see you !")
        return
      pos = target.position
      bot.pathfinder.setMovements(movements)
      bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
    if 'stop' in message:
      off(bot, 'chat', handleMsg)

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [2]:
bot.placeBlock

[AsyncFunction: placeBlock]

In [3]:
mblock = require('./control_primitives/mineBlock.js')
mblock.mineBlock(bot, 'dirt', 1, timeout=5000)

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [4]:
place = require('./control_primitives/placeItem.js')
place.placeItem(bot, 'dirt', Vec3(bot.entity.position.x, bot.entity.position.y+1, bot.entity.position.z), timeout=5000)

{ id: 15, name: 'dirt', displayName: 'Dirt', stackSize: 64 }

Vec3 { x: 17.563921057800393, y: 69, z: -53.51182544587126 }



In [2]:
craft = require('./control_primitives/craftItem.js')
craft.craftItem(bot, 'oak_planks', 1, timeout=5000)

In [3]:
craft = require('./control_primitives/craftItem.js')
craft.craftItem(bot, 'crafting_table', 1, timeout=5000)

In [4]:
place = require('./control_primitives/placeItem.js')
place.placeItem(bot, 'crafting_table', 1, bot.entity.position, timeout=5000)

Vec3 { x: undefined, y: undefined, z: undefined }

☕  JavaScript Error  Call to 'placeItem' failed:
> place.placeItem(bot, 'crafting_table', 1, bot.entity.position, timeout=5000)
  at <module> (C:\Users\g-pc\AppData\Local\Temp\ipykernel_8228\1800787311.py:2)
> 

... across the bridge ...

  at Pipe.onStreamRead (node:internal/stream_base_commons:190:23)
  at Readable.push (node:internal/streams/readable:390:5)
  at readableAddChunkPushByteMode (node:internal/streams/readable:510:3)
  at addChunk (node:internal/streams/readable:559:12)
  at Socket.emit (node:events:518:28)
  at Socket.<anonymous> (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:288:18)
  at Bridge.onMessage (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:231:25)
  at Bridge.call (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:136:42)
  at Object.placeItem (c:\Users\g-pc\Docume

In [ ]:
print(f'{bot.entity.position.x}=')

23.5=


In [ ]:
async def check_if_moving(bot):
    print("checking if moving")
    print(bot.pathfinder.isMoving())
    while bot.pathfinder.isMoving():
        print("moving")
        time.sleep(1)
    return

async def mine_block(bot, name, count=1):
    RANGE_GOAL = 3
    _mine_block_fail_count = 0

    # return if name is not string
    if not isinstance(name, str):
        raise ValueError(f"name for mine_block must be a string, got {name}")
    if not isinstance(count, int):
        raise ValueError(f"count for mine_block must be a number, got {count}")
    block_by_name = mcData.blocksByName[name]
    if block_by_name is None:
        raise ValueError(f"No block named {name}")
    blocks = bot.findBlocks({
        "matching": [block_by_name.id],
        "maxDistance": 64,
        "count": count,
    })
    if len(list(blocks)) == 0:
        bot.chat(f"No {name} nearby, please explore first")
        _mine_block_fail_count += 1
        if _mine_block_fail_count > 10:
            raise ValueError(
                "mine_block failed too many times, make sure you explore before calling mine_block"
            )
        return
    targets = []
    for i in range(len(list(blocks))):
        targets.append(bot.blockAt(blocks[i]))

    for i, target in enumerate(targets):
        print(i)
        goalNear = pathfinder.goals.GoalNear(target.position.x, target.position.y, target.position.z, RANGE_GOAL, timeout=10000)
        print(target.position.x, target.position.y, target.position.z)
        try: 
            bot.pathfinder.goto(goalNear, timeout=10000)
            bot.collectBlock.collect(target, timeout=10000)
        except:
            print("cannot reach!")
            goalY = pathfinder.goals.GoalY(target.position.y)
            bot.pathfinder.goto(goalY)
        print(f'{bot.pathfinder.isMoving()=}')      


In [ ]:
await mine_block(bot, "grass_block", 10)

NameError: name 'mine_block' is not defined

In [ ]:
await mine_block(bot, "oak_log", 3)

0
-8 75 -70
[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

bot.pathfinder.isMoving()=False
1
-9 69 -72
[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

bot.pathfinder.isMoving()=False
2
-9 70 -72
bot.pathfinder.isMoving()=False


In [ ]:
async def craft_item(bot, name, count=1):
    craftingTable = bot.findBlock({
        'matching': mcData.blocksByName['crafting_table'].id
    })
    print(craftingTable)
    
    # Just need to get the proper recipe
    recipe = mcData.recipes[mcData.itemsByName[name].id]

    # Check if the recipe is too big to craft without a crafting table
    try:
        if len(list(recipe.inShape)) > 4 and not craftingTable:
            bot.chat("Too big to craft without crafting table")
            return
    except:
        print("Recipe is shapeless. Able to continue!")
        
    if recipe:
        try:
            bot.craft(recipe, count, craftingTable)
            print("crafting")
            bot.chat(f"Crafted {count} {name}")
        except:
            print("cannot craft")
            return
    else:
        print("no recipe")
        return

 Installing '/control_primitives/mineBlock.js' version 'latest'... This will only happen once. 

[JSE] npm ERR! code EINVALIDPACKAGENAME

[JSE] npm ERR! Invalid package name "/control_primitives/mineBlock.js" of package "/control_primitives/mineBlock.js@latest": name can only contain URL-friendly characters.

[JSE] 

[JSE] npm ERR! A complete log of this run can be found in: C:\Users\g-pc\AppData\Local\npm-cache\_logs\2024-01-31T02_22_38_927Z-debug-0.log

☕  JavaScript Error  Call to 'require' failed:
> mblock = require('/control_primitives/mineBlock.js')
  at <module> (C:\Users\g-pc\AppData\Local\Temp\ipykernel_8216\429911443.py:1)
> 

... across the bridge ...

  at addChunk (node:internal/streams/readable:559:12)
  at Socket.emit (node:events:518:28)
  at Socket.<anonymous> (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:288:18)
  at Bridge.onMessage (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\j

In [ ]:
killMob(bot, "chicken")

☕  JavaScript Error  Call to '' failed:
> killMob(bot, "chicken")
  at <module> (C:\Users\g-pc\AppData\Local\Temp\ipykernel_8216\870129149.py:1)
> 

... across the bridge ...

  at Pipe.onStreamRead (node:internal/stream_base_commons:190:23)
  at Readable.push (node:internal/streams/readable:390:5)
  at readableAddChunkPushByteMode (node:internal/streams/readable:510:3)
  at addChunk (node:internal/streams/readable:559:12)
  at Socket.emit (node:events:518:28)
  at Socket.<anonymous> (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:288:18)
  at Bridge.onMessage (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:231:25)
  at Bridge.call (c:\Users\g-pc\AppData\Local\Programs\Python\Python310\lib\site-packages\javascript\js\bridge.js:138:35)
  TypeError: this.m[ffid] is not a function
> ^
🌉 TypeError: this.m[ffid] is not a function
